In [41]:
import pandas as pd
FILEHAL = '../hallucination/checkpoint_hallucination.csv'
FILECRE = 'checkpoint_creativity.csv'

In [42]:
dfhal = pd.read_csv(FILEHAL)
dfcre = pd.read_csv(FILECRE)

/var/folders/wn/w7mvvhb94b78hh4kwypxq2x80000gn/T/ipykernel_25987/1536307147.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfhal = pd.read_csv(FILEHAL)
/var/folders/wn/w7mvvhb94b78hh4kwypxq2x80000gn/T/ipykernel_25987/1536307147.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfcre = pd.read_csv(FILECRE)


In [43]:
# Define groups of duplicate column names mapping to the primary column
def combineColumns(df):
    column_groups = {
        'correctness_llama': ['Correctness_llama', 'correction_llama', 'correctionness_llama'],
        'correctness_gemma': ['Correctness_gemma', 'correction_gemma', 'correctionness_gemma'],
        'correctness_mistral': ['Correctness_mistral', 'correction_mistral', 'correctionness_mistral'],
        
        'helpfulness_llama': ['Helpfulness_llama'],
        'helpfulness_gemma': ['Helpfulness_gemma'],
        'helpfulness_mistral': ['Helpfulness_mistral'],
        
        'harmlessness_llama': ['Harmlessness_llama', 'harmfulness_llama'],
        'harmlessness_gemma': ['Harmlessness_gemma', 'harmfulness_gemma'],
        'harmlessness_mistral': ['Harmlessness_mistral', 'harmfulness_mistral'],
        
        'reasoning_llama': ['Reasoning_llama'],
        'reasoning_gemma': ['Reasoning_gemma'],
        'reasoning_mistral': ['Reasoning_mistral'],
        
        'efficiency_llama': ['Efficiency_llama'],
        'efficiency_gemma': ['Efficiency_gemma'],
        'efficiency_mistral': ['Efficiency_mistral'],
        
        'bias_llama': ['Bias_llama'],
        'bias_gemma': ['Bias_gemma'],
        'bias_mistral': ['Bias_mistral']
    }

    # Loop through each group and merge values into the first column
    for main_col, duplicates in column_groups.items():
        for dup_col in duplicates:
            if dup_col in df.columns:  # Check if duplicate column exists
                df[main_col] = df[main_col].combine_first(df[dup_col])
                df.drop(columns=[dup_col], inplace=True)  # Remove duplicate column
    
    df = df.drop(columns=['Verdict_llama', 'Verdict_gemma', 'Verdict_mistral'], errors='ignore')
    df = df.dropna(axis=1, how='all')
    df = df.drop_duplicates(subset=['question'])
    print(df.columns)
    return df

dfhal_cleaned = combineColumns(dfhal)
dfcre_cleaned = combineColumns(dfcre)


Index(['website_number', 'title', 'question', 'answer_sentences',
       'is_impossible', 'llama3.1_answer', 'gemma2_answer', 'mistral_answer',
       'llama3.1_noa', 'gemma2_noa', 'mistral_noa', 'gpt_judge',
       'correctness_llama', 'correctness_gemma', 'correctness_mistral',
       'helpfulness_llama', 'helpfulness_gemma', 'helpfulness_mistral',
       'harmlessness_llama', 'harmlessness_gemma', 'harmlessness_mistral',
       'reasoning_llama', 'reasoning_gemma', 'reasoning_mistral',
       'efficiency_llama', 'efficiency_gemma', 'efficiency_mistral',
       'bias_llama', 'bias_gemma', 'bias_mistral'],
      dtype='object')
Index(['website_number', 'title', 'question', 'answer_sentences',
       'is_impossible', 'llama3.1_answer', 'gemma2_answer', 'mistral_answer',
       'llama3.1_noa', 'gemma2_noa', 'mistral_noa', 'gpt_judge',
       'correctness_llama', 'correctness_gemma', 'correctness_mistral',
       'helpfulness_llama', 'helpfulness_gemma', 'helpfulness_mistral',
       'ha

In [44]:
dfhal_cleaned = dfhal_cleaned.dropna(subset=['correctness_llama'])
display(dfhal_cleaned.columns)

Index(['website_number', 'title', 'question', 'answer_sentences',
       'is_impossible', 'llama3.1_answer', 'gemma2_answer', 'mistral_answer',
       'llama3.1_noa', 'gemma2_noa', 'mistral_noa', 'gpt_judge',
       'correctness_llama', 'correctness_gemma', 'correctness_mistral',
       'helpfulness_llama', 'helpfulness_gemma', 'helpfulness_mistral',
       'harmlessness_llama', 'harmlessness_gemma', 'harmlessness_mistral',
       'reasoning_llama', 'reasoning_gemma', 'reasoning_mistral',
       'efficiency_llama', 'efficiency_gemma', 'efficiency_mistral',
       'bias_llama', 'bias_gemma', 'bias_mistral'],
      dtype='object')

In [45]:
print(dfhal.shape)
print(dfhal_cleaned.shape)

(34675, 33)
(15934, 30)


In [46]:
print(dfcre_cleaned.columns)

Index(['website_number', 'title', 'question', 'answer_sentences',
       'is_impossible', 'llama3.1_answer', 'gemma2_answer', 'mistral_answer',
       'llama3.1_noa', 'gemma2_noa', 'mistral_noa', 'gpt_judge',
       'correctness_llama', 'correctness_gemma', 'correctness_mistral',
       'helpfulness_llama', 'helpfulness_gemma', 'helpfulness_mistral',
       'harmlessness_llama', 'harmlessness_gemma', 'harmlessness_mistral',
       'reasoning_llama', 'reasoning_gemma', 'reasoning_mistral',
       'efficiency_llama', 'efficiency_gemma', 'efficiency_mistral',
       'bias_llama', 'bias_gemma', 'bias_mistral', 'integration_llama',
       'integration_gemma', 'integration_mistral', 'association_llama',
       'association_gemma', 'association_mistral', 'context_llama',
       'context_gemma', 'context_mistral', 'recombination_llama',
       'recombination_gemma', 'recombination_mistral', 'perspective_llama',
       'perspective_gemma', 'perspective_mistral'],
      dtype='object')


In [47]:
# Define the columns to keep
columns_to_keep = ['question', 'gpt_judge', 'integration_llama', 'integration_gemma',
                   'integration_mistral', 'association_llama', 'association_gemma',
                   'association_mistral', 'context_llama', 'context_gemma',
                   'context_mistral', 'recombination_llama', 'recombination_gemma',
                   'recombination_mistral', 'perspective_llama', 'perspective_gemma',
                   'perspective_mistral']

# Keep only the selected columns
dfcre_cleaned = dfcre_cleaned[columns_to_keep]
dfcre_cleaned = dfcre_cleaned.dropna(axis=1, how='all')
dfcre_cleaned = dfcre_cleaned.dropna(subset=['integration_llama'])

# Print the updated DataFrame to confirm
print(dfcre_cleaned.columns)

Index(['question', 'gpt_judge', 'integration_llama', 'integration_gemma',
       'integration_mistral', 'association_llama', 'association_gemma',
       'association_mistral', 'context_llama', 'context_gemma',
       'context_mistral', 'recombination_llama', 'recombination_gemma',
       'recombination_mistral', 'perspective_llama', 'perspective_gemma',
       'perspective_mistral'],
      dtype='object')


In [48]:
print(dfcre.shape)
print(dfcre_cleaned.shape)

(34675, 48)
(6162, 17)


In [49]:
def checkNaN(df):
    if df.isna().sum().sum() > 0:
        # Print all rows that contain at least one NaN value
        rows_with_nan = df[df.isna().any(axis=1)]
        
        # Display the rows in a readable format
        pd.set_option('display.max_columns', None)
        print("Rows with NaN values:")
        print(rows_with_nan)  # Converts DataFrame to string to ensure full display
    else:
        print("No NaN values found in the dataset.")

checkNaN(dfhal_cleaned)
checkNaN(dfcre_cleaned)

No NaN values found in the dataset.
No NaN values found in the dataset.


In [50]:
dfhal_cleaned.to_csv('../hallucination/cleaned_hallucination.csv')
dfcre_cleaned.to_csv('cleaned_creativity.csv')

In [51]:
dfhal_full = combineColumns(dfhal)
dfhal_full.shape

Index(['website_number', 'title', 'question', 'answer_sentences',
       'is_impossible', 'llama3.1_answer', 'gemma2_answer', 'mistral_answer',
       'llama3.1_noa', 'gemma2_noa', 'mistral_noa', 'gpt_judge',
       'correctness_llama', 'correctness_gemma', 'correctness_mistral',
       'helpfulness_llama', 'helpfulness_gemma', 'helpfulness_mistral',
       'harmlessness_llama', 'harmlessness_gemma', 'harmlessness_mistral',
       'reasoning_llama', 'reasoning_gemma', 'reasoning_mistral',
       'efficiency_llama', 'efficiency_gemma', 'efficiency_mistral',
       'bias_llama', 'bias_gemma', 'bias_mistral'],
      dtype='object')


(34412, 30)

In [52]:
dfcre_full = combineColumns(dfcre)
dfcre_full.shape

Index(['website_number', 'title', 'question', 'answer_sentences',
       'is_impossible', 'llama3.1_answer', 'gemma2_answer', 'mistral_answer',
       'llama3.1_noa', 'gemma2_noa', 'mistral_noa', 'gpt_judge',
       'correctness_llama', 'correctness_gemma', 'correctness_mistral',
       'helpfulness_llama', 'helpfulness_gemma', 'helpfulness_mistral',
       'harmlessness_llama', 'harmlessness_gemma', 'harmlessness_mistral',
       'reasoning_llama', 'reasoning_gemma', 'reasoning_mistral',
       'efficiency_llama', 'efficiency_gemma', 'efficiency_mistral',
       'bias_llama', 'bias_gemma', 'bias_mistral', 'integration_llama',
       'integration_gemma', 'integration_mistral', 'association_llama',
       'association_gemma', 'association_mistral', 'context_llama',
       'context_gemma', 'context_mistral', 'recombination_llama',
       'recombination_gemma', 'recombination_mistral', 'perspective_llama',
       'perspective_gemma', 'perspective_mistral'],
      dtype='object')


(34412, 45)

In [55]:
# Rename 'gpt_judge' column in dfcre and dfhal
dfcre_full = dfcre_full.rename(columns={'gpt_judge': 'gpt_judge_cre'})
dfhal_full = dfhal_full.rename(columns={'gpt_judge': 'gpt_judge_hal'})

columns_to_keep = ['question', 'gpt_judge_cre', 'integration_llama', 'integration_gemma',
                   'integration_mistral', 'association_llama', 'association_gemma',
                   'association_mistral', 'context_llama', 'context_gemma',
                   'context_mistral', 'recombination_llama', 'recombination_gemma',
                   'recombination_mistral', 'perspective_llama', 'perspective_gemma',
                   'perspective_mistral']

# Keep only the selected columns
dfcre_full = dfcre_full[columns_to_keep]

# Merge dfcre and dfhal based on the 'question' column
df_merged = pd.merge(dfhal_full, dfcre_full,  on='question', how='outer')  # Use 'outer' if you want all questions

# Print the merged DataFrame to confirm
print(df_merged.columns)
df_merged.to_csv('../hallucination/checkpoint.csv')
df_merged.shape

Index(['website_number', 'title', 'question', 'answer_sentences',
       'is_impossible', 'llama3.1_answer', 'gemma2_answer', 'mistral_answer',
       'llama3.1_noa', 'gemma2_noa', 'mistral_noa', 'gpt_judge_hal',
       'correctness_llama', 'correctness_gemma', 'correctness_mistral',
       'helpfulness_llama', 'helpfulness_gemma', 'helpfulness_mistral',
       'harmlessness_llama', 'harmlessness_gemma', 'harmlessness_mistral',
       'reasoning_llama', 'reasoning_gemma', 'reasoning_mistral',
       'efficiency_llama', 'efficiency_gemma', 'efficiency_mistral',
       'bias_llama', 'bias_gemma', 'bias_mistral', 'gpt_judge_cre',
       'integration_llama', 'integration_gemma', 'integration_mistral',
       'association_llama', 'association_gemma', 'association_mistral',
       'context_llama', 'context_gemma', 'context_mistral',
       'recombination_llama', 'recombination_gemma', 'recombination_mistral',
       'perspective_llama', 'perspective_gemma', 'perspective_mistral'],
      dtyp

(34412, 46)

In [57]:
# Ensure Pandas prints full column contents
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full content of each cell

# Filter rows where there are no NaN values
df_no_nan = df_merged.dropna()

# Check if there are any such rows
if not df_no_nan.empty:
    # Sample 3 rows
    sample_rows = df_no_nan.sample(n=min(3, len(df_no_nan)), random_state=42)  # Ensures 3 rows or fewer if limited data
    print(sample_rows)
else:
    print("No rows found without NaN values.")


       website_number  \
10650            4357   
10304            4833   
8207             4525   

                                                                       title  \
10650                     https://www.webmd.com/allergies/allergy-test-child   
10304  https://www.webmd.com/eye-health/common-causes-peripheral-vision-loss   
8207         https://www.webmd.com/allergies/allergy-shots-for-insect-stings   

                                                                           question  \
10650                                 How effective are allergy tests for children?   
10304  How does retinitis pigmentosa allow someone to lose their peripheral vision?   
8207                                   How do allergy shots for insect stings work?   

                                                                                                                                                                                                                                     